In [13]:
%%writefile mm1.cpp
#include <stdio.h>
#include <stdlib.h>


int SIZE;

#define _first_(i,j) first[ (j)*SIZE + (i) ]
#define _second_(i,j) second[ (j)*SIZE + (i) ]
#define _multiply_(i,j) multiply[ (j)*SIZE + (i) ]

#include <sys/time.h>
#include <time.h>

static double gtod_ref_time_sec = 0.0;

/* Adapted from the bl2_clock() routine in the BLIS library */

double dclock()
{
  double         the_time, norm_sec;
  struct timeval tv;
  gettimeofday( &tv, NULL );
  if ( gtod_ref_time_sec == 0.0 )
    gtod_ref_time_sec = ( double ) tv.tv_sec;
  norm_sec = ( double ) tv.tv_sec - gtod_ref_time_sec;
  the_time = norm_sec + tv.tv_usec * 1.0e-6;
  return the_time;
}

int mm(double * first, double * second, double * multiply)
{
  register unsigned int i,j,k;
  double sum = 0;

  for (i = 0; i < SIZE; i++) { //rows in multiply
    for (j = 0; j < SIZE; j++) { //columns in multiply
      sum = 0;
      for (k = 0; k < SIZE; k++) { //columns in first and rows in second
            sum = sum + _first_(i,k)*_second_(k,j);
          }
      _multiply_(i,j) = sum;
    }
  }
  return 0;
}

int main( int argc, const char* argv[] )
{
  int i,j,iret;
  double ** first;
  double ** second;
  double ** multiply;

  double * first_;
  double * second_;
  double * multiply_;

  double dtime;

  SIZE = 1000;

  first_ = (double*) malloc(SIZE*SIZE*sizeof(double));
  second_ = (double*) malloc(SIZE*SIZE*sizeof(double));
  multiply_ = (double*) malloc(SIZE*SIZE*sizeof(double));

  first = (double**) malloc(SIZE*sizeof(double*));
  second = (double**) malloc(SIZE*sizeof(double*));
  multiply = (double**) malloc(SIZE*sizeof(double*));

  for (i = 0; i < SIZE; i++) {
    first[i] = first_ + i*SIZE;
    second[i] = second_ + i*SIZE;
    multiply[i] = multiply_ + i*SIZE;
  }

  for (i = 0; i < SIZE; i++) { //rows in first
    for (j = 0; j < SIZE; j++) { //columns in first
      first[i][j]=i+j;
      second[i][j]=i-j;
    }
  }

  dtime = dclock();

  iret=mm(first_,second_,multiply_);

  dtime = dclock()-dtime;
  printf( "Time: %le \n", dtime);

  fflush( stdout );

  free(first_);
  free(second_);
  free(multiply_);

  free(first);
  free(second);
  free(multiply);

  return iret;
}

Overwriting mm1.cpp


In [15]:
!g++ -fopenmp mm1.cpp -o mm1

mm1.cpp: In function ‘int mm(double*, double*, double*)’:
mm1.cpp:32:25: warning: ISO C++17 does not allow ‘register’ storage class specifier []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wregister-Wregister]8;;]
   32 |   register unsigned int i,j,k;
      |                         ^
mm1.cpp:32:27: warning: ISO C++17 does not allow ‘register’ storage class specifier []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wregister-Wregister]8;;]
   32 |   register unsigned int i,j,k;
      |                           ^
mm1.cpp:32:29: warning: ISO C++17 does not allow ‘register’ storage class specifier []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wregister-Wregister]8;;]
   32 |   register unsigned int i,j,k;
      |                             ^


In [16]:
!./mm1

Time: 5.998810e+00 


In [17]:
! OMP_NUM_THREADS=4 ./mm1

Time: 6.116760e+00 


In [28]:
%%writefile prog.cpp

#include<stdio.h>
#include<omp.h>

int main(){
  printf("START\n");
  #pragma omp parallel
  {
    int idx = omp_get_thread_num();
    printf("Watek %d\n", idx);
    #pragma omp single
    printf("Single %d\n", idx);
    #pragma omp sections
    {
      #pragma omp section
      printf("Sekcja 1 %d\n", idx);
      #pragma omp section
      printf("Sekcja 2 %d\n", idx);
    }
  }
  printf("KONIEC\n");
  return 0;
}


Overwriting prog.cpp


In [30]:
! g++ -fopenmp prog.cpp -o prog && OMP_NUM_THREADS=2 ./prog

START
Watek 0
Single 0
Watek 1
Sekcja 1 0
Sekcja 2 1
KONIEC


In [41]:
%%writefile prog2.cpp

#include<stdio.h>
#include<omp.h>

int main(){
  printf("START\n");
  int suma = 0;
  #pragma omp parallel
  {
    int idx = omp_get_thread_num();
    printf("Watek %d\n", idx);
    #pragma omp for reduction(+:suma)
    for(int i=0;i<12;i++){
      suma++;
      printf("Watek %d i = %d suma=%d\n",idx,i,suma);
    }
    printf("Watek %d suma %d \n", idx, suma);
  }
  printf("suma = %d\nKONIEC\n",suma);
  return 0;
}


Overwriting prog2.cpp


In [42]:
! g++ -fopenmp prog2.cpp -o prog2 && OMP_NUM_THREADS=4 ./prog2

START
Watek 3
Watek 3 i = 9 suma=1
Watek 3 i = 10 suma=2
Watek 3 i = 11 suma=3
Watek 2
Watek 2 i = 6 suma=1
Watek 2 i = 7 suma=2
Watek 2 i = 8 suma=3
Watek 0
Watek 0 i = 0 suma=1
Watek 0 i = 1 suma=2
Watek 0 i = 2 suma=3
Watek 1
Watek 1 i = 3 suma=1
Watek 1 i = 4 suma=2
Watek 1 i = 5 suma=3
Watek 1 suma 12 
Watek 2 suma 12 
Watek 3 suma 12 
Watek 0 suma 12 
suma = 12
KONIEC


# Zadania

## 1

In [59]:
%%writefile task1.cpp

#include <stdio.h>
#include <omp.h>

#define N 1000000

void scalar_vector_mul(double alpha, double *v, double *result) {
    #pragma omp parallel for
    for(int i = 0; i < N; i++) {
        result[i] = alpha * v[i];
    }
}


double dot_product(double *w, double *v) {
    double sum = 0.0;
    #pragma omp parallel for reduction(+:sum)
    for(int i = 0; i < N; i++) {
        sum += w[i] * v[i];
    }
    return sum;
}

int main() {
    double alpha = 2.0;
    double v[N], w[N], result[N];

    for(int i = 0; i < N; i++){
        v[i] = 1;
        w[i] = 2;
    }

    scalar_vector_mul(alpha, v, result);
    double dot = dot_product(w, v);

    printf("Iloczyn skalarny= %f\n", dot);

    return 0;
}


Overwriting task1.cpp


In [60]:
!g++ task1.cpp -fopenmp -o task1 && OMP_NUM_THREADS=4 ./task1

Iloczyn skalarny= 2000000.000000


##2

In [83]:
%%writefile mm1.cpp
#include <stdio.h>
#include <stdlib.h>


int SIZE;

#define _first_(i,j) first[ (j)*SIZE + (i) ]
#define _second_(i,j) second[ (j)*SIZE + (i) ]
#define _multiply_(i,j) multiply[ (j)*SIZE + (i) ]

#include <sys/time.h>
#include <time.h>

static double gtod_ref_time_sec = 0.0;

/* Adapted from the bl2_clock() routine in the BLIS library */

double dclock()
{
  double         the_time, norm_sec;
  struct timeval tv;
  gettimeofday( &tv, NULL );
  if ( gtod_ref_time_sec == 0.0 )
    gtod_ref_time_sec = ( double ) tv.tv_sec;
  norm_sec = ( double ) tv.tv_sec - gtod_ref_time_sec;
  the_time = norm_sec + tv.tv_usec * 1.0e-6;
  return the_time;
}

int mm(double * first, double * second, double * multiply)
{
  register unsigned int i,j,k;
  double sum = 0;

  for (i = 0; i < SIZE; i++) { //rows in multiply
    for (j = 0; j < SIZE; j++) { //columns in multiply
      sum = 0;
      for (k = 0; k < SIZE; k++) { //columns in first and rows in second
            sum = sum + _first_(i,k)*_second_(k,j);
          }
      _multiply_(i,j) = sum;
    }
  }
  return 0;
}

int main( int argc, const char* argv[] )
{
  int i,j,iret;
  double ** first;
  double ** second;
  double ** multiply;

  double * first_;
  double * second_;
  double * multiply_;

  double dtime;

  SIZE = 1000;

  first_ = (double*) malloc(SIZE*SIZE*sizeof(double));
  second_ = (double*) malloc(SIZE*SIZE*sizeof(double));
  multiply_ = (double*) malloc(SIZE*SIZE*sizeof(double));

  first = (double**) malloc(SIZE*sizeof(double*));
  second = (double**) malloc(SIZE*sizeof(double*));
  multiply = (double**) malloc(SIZE*sizeof(double*));

  for (i = 0; i < SIZE; i++) {
    first[i] = first_ + i*SIZE;
    second[i] = second_ + i*SIZE;
    multiply[i] = multiply_ + i*SIZE;
  }

  for (i = 0; i < SIZE; i++) { //rows in first
    for (j = 0; j < SIZE; j++) { //columns in first
      first[i][j]=i+j;
      second[i][j]=i-j;
    }
  }

  dtime = dclock();

  iret=mm(first_,second_,multiply_);

  dtime = dclock()-dtime;
  printf( "Time: %le \n", dtime);

  fflush( stdout );

  free(first_);
  free(second_);
  free(multiply_);

  free(first);
  free(second);
  free(multiply);

  return iret;
}


Overwriting mm1.cpp


In [84]:
!g++ mm1.cpp -o mm1 && ./mm1

mm1.cpp: In function ‘int mm(double*, double*, double*)’:
mm1.cpp:32:25: warning: ISO C++17 does not allow ‘register’ storage class specifier []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wregister-Wregister]8;;]
   32 |   register unsigned int i,j,k;
      |                         ^
mm1.cpp:32:27: warning: ISO C++17 does not allow ‘register’ storage class specifier []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wregister-Wregister]8;;]
   32 |   register unsigned int i,j,k;
      |                           ^
mm1.cpp:32:29: warning: ISO C++17 does not allow ‘register’ storage class specifier []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wregister-Wregister]8;;]
   32 |   register unsigned int i,j,k;
      |                             ^
Time: 5.573328e+00 


In [96]:
%%writefile mm2.cpp
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

int SIZE;

#define _first_(i,j) first[ (j)*SIZE + (i) ]
#define _second_(i,j) second[ (j)*SIZE + (i) ]
#define _multiply_(i,j) multiply[ (j)*SIZE + (i) ]

#include <sys/time.h>
#include <time.h>

static double gtod_ref_time_sec = 0.0;

/* Adapted from the bl2_clock() routine in the BLIS library */

double dclock()
{
  double         the_time, norm_sec;
  struct timeval tv;
  gettimeofday( &tv, NULL );
  if ( gtod_ref_time_sec == 0.0 )
    gtod_ref_time_sec = ( double ) tv.tv_sec;
  norm_sec = ( double ) tv.tv_sec - gtod_ref_time_sec;
  the_time = norm_sec + tv.tv_usec * 1.0e-6;
  return the_time;
}

int mm(double * first, double * second, double * multiply)
{
  unsigned int i,j,k;
  double sum = 0;

  #pragma omp parallel for private(j,k,sum)
  for (i = 0; i < SIZE; i++) { //rows in multiply
    for (j = 0; j < SIZE; j++) { //columns in multiply
      sum = 0;
      for (k = 0; k < SIZE; k++) { //columns in first and rows in second
            sum = sum + _first_(i,k)*_second_(k,j);
          }
      _multiply_(i,j) = sum;
    }
  }
  return 0;
}

int main( int argc, const char* argv[] )
{
  int i,j,iret;
  double ** first;
  double ** second;
  double ** multiply;

  double * first_;
  double * second_;
  double * multiply_;

  double dtime;

  SIZE = 1000;

  first_ = (double*) malloc(SIZE*SIZE*sizeof(double));
  second_ = (double*) malloc(SIZE*SIZE*sizeof(double));
  multiply_ = (double*) malloc(SIZE*SIZE*sizeof(double));

  first = (double**) malloc(SIZE*sizeof(double*));
  second = (double**) malloc(SIZE*sizeof(double*));
  multiply = (double**) malloc(SIZE*sizeof(double*));

  for (i = 0; i < SIZE; i++) {
    first[i] = first_ + i*SIZE;
    second[i] = second_ + i*SIZE;
    multiply[i] = multiply_ + i*SIZE;
  }
  #pragma omp parallel for private(j)
  for (i = 0; i < SIZE; i++) { //rows in first
    for (j = 0; j < SIZE; j++) { //columns in first
      first[i][j]=i+j;
      second[i][j]=i-j;
    }
  }

  dtime = dclock();

  iret=mm(first_,second_,multiply_);

  dtime = dclock()-dtime;
  printf( "Time: %le \n", dtime);

  fflush( stdout );

  free(first_);
  free(second_);
  free(multiply_);

  free(first);
  free(second);
  free(multiply);

  return iret;
}


Overwriting mm2.cpp


In [98]:
!g++ mm2.cpp -fopenmp -o mm2 && OMP_NUM_THREADS=2 ./mm2

Time: 8.944514e+00 
Threads used: 2


In [93]:
import multiprocessing
print(multiprocessing.cpu_count())

2


3

In [109]:
%%writefile task3.cpp
#include <iostream>
#include <cmath>
#include <omp.h>

using namespace std;

double func(double x) {
    return sin(x) * x * x;
}

double trapezoidalRule(double (*f)(double), double a, double b, int n) {
    double h = (b - a) / n;
    double sum = 0.0;

    #pragma omp parallel for reduction(+:sum)
    for (int i = 1; i < n; i++) {
        double x = a + i * h;
        sum += f(x);
    }

    double result = (h / 2.0) * (f(a) + 2.0 * sum + f(b));

    return result;
}

double trapezoidalRuleEpsilon(double (*f)(double), double a, double b, double epsilon) {
    int n = 10;
    double prev_result = 0.0;
    double result = trapezoidalRule(f, a, b, n);

    while (fabs(result - prev_result) > epsilon) {
        prev_result = result;
        n *= 2;
        result = trapezoidalRule(f, a, b, n);
    }
    return result;
}

int main() {
    double a = 0.0;
    double b = M_PI;
    int n = 1000000;

    double start_time = omp_get_wtime();
    double integral = trapezoidalRule(func, a, b, n);

    double end_time = omp_get_wtime();

    std::cout << "Całka z sin(x)*x^2 na przedziale [" << a << ", " << b << "]" << std::endl;
    std::cout << "Wynik (n=" << n << "): " << integral << std::endl;
    std::cout <<(end_time - start_time) << " sekund" << std::endl;

    double epsilon = 1e-6;
    start_time = omp_get_wtime();

    double integral_eps = trapezoidalRuleEpsilon(func, a, b, epsilon);

    end_time = omp_get_wtime();

    std::cout << "Wynik (epsilon=" << epsilon << "): " << integral_eps << std::endl;
    std::cout << (end_time - start_time) << " sekund" << std::endl;

    return 0;
}

Overwriting task3.cpp


In [111]:
!g++ -fopenmp -o task3 task3.cpp && OMP_NUM_THREADS=1 ./task3

Całka z sin(x)*x^2 na przedziale [0, 3.14159]
Wynik (n=1000000): 5.8696
0.0218426 sekund
Wynik (epsilon=1e-06): 5.8696
0.000253637 sekund


In [110]:
!g++ -fopenmp -o task3 task3.cpp && OMP_NUM_THREADS=2 ./task3

Całka z sin(x)*x^2 na przedziale [0, 3.14159]
Wynik (n=1000000): 5.8696
0.0356173 sekund
Wynik (epsilon=1e-06): 5.8696
0.0397335 sekund
